## Entity Matching
    
    
Let's consider that you, as a data scientist, are working for **Retailer A**, a large-scale software products chain. Retailer A has recently entered into a strategic partnership with **Retailer B**, an online software product platform. As part of this partnership, Retailer B has shared its product dataset with Retailer A for the purpose of cross-promotion, product indexing and targeted marketing.

Your task is to perform **entity resolution**, also known as record linkage, on these datasets. The goal is to identify which products in Retailer B's dataset are also products of Retailer A. This will allow the marketing department to create more personalized product offering campaigns and product indexes.

Entity resolution can be a complex task due to various reasons such as data inconsistencies, missing values, and the need to protect customer privacy. It involves several steps including data cleaning, data standardization, and matching records.

You may find the datasets in CSV format:

    - DDB_retailerA.csv
    - DDB_retailerB.csv
    
Notes:

- Schema aligned
- Structured data (no large mess ups, void cells, etc ...)
- Blocking already performed

### Entity resolution

Data stored in information systems are often erroneous. The
most typical errors include: inconsistent, missing, and outdated
values, typos as well as duplicates. To handle data of poor quality,
data cleaning (a.k.a. curation) and deduplication (a.k.a. entity
resolution) methods are used in projects realized by research
and industry. This is of particular challenge due
to its computational complexity and the complexity of finding
the most adequate method for comparing records and computing
similarities of these records. The similarity value of two records
is a compound value, whose computation is based on similarities
of individual attribute values.


Recall that an ER pipeline includes four basic tasks, namely:
    
- Blocking (a.k.a. indexing) - it organizes records into groups,
such that each group includes records that may include
potential duplicates.

- Block processing (a.k.a. filtering) - its goal is to eliminate
records that do not have to be compared.

- **Entity matching** (a.k.a. similarity computation) - it computes similarity values between records compared in pairs,
i.e., a value of each attribute in one record is compared to
a value of a corresponding attribute in the second record.

- **Entity clustering** - it aims at creating groups of similar records, from pairs of records representing highly probable duplicates.


For simplicity, we are going to focus on entity matching and entity clustering. This is a fairly basic pipeline and it can be extended in many ways. For example, sophisticated pre-processing and matching algorithms can be used. Or thinking about how to scale the entity resolution (i.e. speeding up the process without sacrificying accuracy).



## Deliverable format and tests

To evaluate the assignment,we will use a python script (*ddb_entity_matching.py*) that takes a data folder containing 2 CSV datasets and outputs a single CSV file called "match_report.csv" with the format below. You may use jupyter notebook to play with you algortihmic solution or other entity matching tools. However, the final deliverable will contain the aforementioned script, this jupyter notebook and requirements.txt file with all the libraries.

It should work this way: python ddb_entity_matching.py -i retailer_data_folder/ -o match_report_file_path

- OUTPUT SCHEMA:

| retailerA_id | retailerB_id | match |   |   |
|--------------|--------------|-------|---|---|
| 571          | 946          | 0     |   |   |
| 574          | 2423         | 0     |   |   |
| 250          | 2839         | 0     |   |   |
| 1162         | 2109         | 1     |   |   |


## Step 1: Data cleaning

Perform data cleaning with the help of regex expressions, removal of stopwords and the tokenization of the documents/sentences or paragraphs. Things like casing, extra spaces, quotes and new lines can be ignored (not limited to that though).

spaCy provides a one-stop-shop for tasks commonly used in any NLP project, so they might have all these functionalities. It useful to explore other packages like NTLK, or build the cleaning steps yourself.


The recommended workflow is the following: Read in the CSV files and apply the data cleaning operations.

***Hint***: https://www.nltk.org/api/nltk.tokenize.html

In [5]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text):
    # Convert to lowercase
    text = str(text).lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

def load_and_clean_data(file_path, name_column):
    df = pd.read_csv(file_path)
    # Clean the product name column
    df['product_name_clean'] = df[name_column].apply(clean_text)
    return df

# Load and clean datasets
df_a = load_and_clean_data('DDB_retailerA.csv', 'name')
df_b = load_and_clean_data('DDB_retailerB.csv', 'title')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Step 2: Entity Matching

For every product record in the retailer A, we need to compute its similarity to every record in retailer B- Think about the total number of comparisons you need to perform.

A very simple approach would be to use rule matching based on matches from regular expressions.Different measures can be used to process different attribute blocks, like using string a similarity metric like Levenshtein distance for names or Jaccard similarity to compare associated groups (say lists of friends on a social network, or a list of investments).

Very powerful techniques can also arise from fuzzy matching and string distance algorithms. One could also try, for example, computing text similarity among sentences/documents/paragraphs using different model or feature extractors, and then compute similarities based on vector distances (i.e. cosine,euclidean,jaccard ) similarity. You may use the following model (not restricted to though):

- **Bag of Words (BoW)** Scikit-Learn, NTLK
- **N-grams**: Scikit-Learn, NTLK
- **TF-IDF**: Scikit-Learn, NTLK
- **Word embedding models**: Word2Vec ( Spacy, Gensim are packages that have this functionality)
- **Pre-trained language models**: BERT and large language models

You could even feed these to an LLM (a small OSS model?). Think of computational cost! Either way, you’ll have to decide on a threshold for creating a link or not as well.


**Extra**: There are ways to avoid this large number of comparisons among entities, they involve sophisticated methods like inserting blocking pipelines. Research this area, only after you have completed all the steps, and as an. You can opt to save the intermediary files (i.e. pairwise similarity matrix) to disk to avoid computing this step every single time. There are also additional open source libraries in python that you may use for the end-to-end entity resolution or parts of it, exclusively dedicated  to ER: *RecordLinkage*, *dedupe*, *Zingg*.

The scale of the dataset is also not expected to demand this level of complexity.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_a = vectorizer.fit_transform(df_a['product_name_clean'])
tfidf_b = vectorizer.transform(df_b['product_name_clean'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_a, tfidf_b)

# Apply threshold for matches (e.g., 0.8)
threshold = 0.8
matches = np.where(cosine_sim >= threshold, 1, 0)

# Generate match report
match_report = []
for i in range(matches.shape[0]):
    for j in range(matches.shape[1]):
        if matches[i][j] == 1:
            match_report.append({
                'retailerA_id': df_a.iloc[i]['id'],
                'retailerB_id': df_b.iloc[j]['id'],
                'match': 1
            })

## Step 3: Evaluation


The file "matching_labels.csv" contains the matched pairs, a value of 1/0 represents a true/false matched pair. You may use this to gauge the performance of your solution by identyfying hits and misses, and precision and recall. These only represent about 3/5 of the total pairs, so be aware of overfitting your solution as the rest will be used to grade your solution. Precision and recall will be used to evaluate your solution.

In [7]:
# Load ground truth labels
true_labels = pd.read_csv('matching_labels.csv')

# Convert predictions to DataFrame
predicted_labels = pd.DataFrame(match_report)

# Calculate precision and recall
true_positives = len(pd.merge(true_labels, predicted_labels, on=['retailerA_id', 'retailerB_id']))
false_positives = len(predicted_labels) - true_positives
false_negatives = len(true_labels) - true_positives

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

print(f'Precision: {precision:.2f}, Recall: {recall:.2f}')

Precision: 0.61, Recall: 0.09


## Step 4: Entity clustering

Once entities have been resolved as matches. we need to  merge them in order to produce a single representative record. This involves grouping the entities based on the similarity scores. Entities or records in the same cluster are considered to be the same.

*Hint: You can use different clustering algorithms, but graph clustering algorithms are the most suitable approach. Think that the output of the entity mathcing stage can be a similarity graph or matrix, you may use tools such as networkX package to apply the needed transformations*



In [8]:
# Implement a clustering procedure that identifies and groups all similar records 
import networkx as nx

# Create similarity graph
G = nx.Graph()
for pair in match_report:
    G.add_edge(pair['retailerA_id'], pair['retailerB_id'])

# Find connected components (clusters)
clusters = list(nx.connected_components(G))

# Save clusters to DataFrame (example)
cluster_report = []
for cluster_id, cluster in enumerate(clusters):
    for node in cluster:
        cluster_report.append({
            'entity_id': node,
            'cluster_id': cluster_id
        })

 Generate Output

In [9]:
pd.DataFrame(match_report).to_csv('match_report.csv', index=False)